In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk

In [2]:
#prepocessing
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()


tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}



def preprocessing(sentence):
    sent = tokenizer.tokenize(sentence.lower().strip())
    tags = nltk.pos_tag(sent)


    sent = ([lemmatizer.lemmatize(item[0], tag_dict.get(item[1][0], wordnet.NOUN)) for item in tags])
    return " ".join(sent)



In [ ]:
#get and clean the training data
df = pd.read_csv("IMDB_dataset.csv")
df['sentiment'].replace("positive",1 ,inplace=True)
df['sentiment'].replace("negative", 0,inplace=True)
df["review"] = df["review"].apply(lambda x: preprocessing(x))
X = df["review"]
Y= df["sentiment"]


In [37]:
from sklearn.svm import LinearSVC
import numpy as np
class NBSVM:
    def __init__(self):
        self.model = None
        self.r = None
    def fit(self,Xtrain,ytrain,C=1):
        p = Xtrain.T.dot(ytrain) +1
        invY = 1 - ytrain
        q = Xtrain.T.dot(invY) + 1
        self.r = np.log((p/np.linalg.norm(p,ord=1)) / (q/np.linalg.norm(q,ord=1)) )

        self.model = LinearSVC(C=C,max_iter=10000)
        Xnb = Xtrain.multiply(self.r)
        self.model.fit(Xnb,ytrain)
    def predict(self,Xtest):
        Xnb = Xtest.multiply(self.r)
        preds = self.model.predict(Xnb)
        return preds

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
vectorizer = CountVectorizer(binary=False,ngram_range=(1,3))
X_train = vectorizer.fit_transform(X_train) 
X_test = vectorizer.transform(X_test) 

In [39]:
clf = NBSVM()
clf.fit(X_train,y_train)
preds = clf.predict(X_test)

In [40]:
actual = np.array(y_test) 

correct = 0
for i in range(len(preds)):
    if preds[i] == actual[i]:
        correct +=1
print(correct/len(preds))

0.91528
